<a href="https://colab.research.google.com/github/ZafraCandia/Amigosecreto/blob/main/Enfoque_en_el_An_lisis_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import sys
import textwrap
from pathlib import Path

try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
except Exception:
    print("Instalando dependencias...")
    !pip install pandas matplotlib --quiet
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np

plt.rcParams['figure.figsize'] = (10,6)

In [21]:
REPO_URL = 'https://github.com/alura-es-cursos/challenge1-data-science-latam.git'
DATA_SUBPATH = 'base-de-datos-challenge1-latam'

if not Path(DATA_SUBPATH).exists():
    print('Clonando repositorio con los datos...')
    os.system(f'git clone {REPO_URL} --depth 1')
else:
    print('La carpeta de datos ya existe. Continuando...')

DATA_DIR = Path(DATA_SUBPATH)
if not DATA_DIR.exists():
    raise FileNotFoundError(f'No se encontró la carpeta de datos en {DATA_DIR}. Revisa que la clonación haya funcionado.')

Clonando repositorio con los datos...


FileNotFoundError: No se encontró la carpeta de datos en base-de-datos-challenge1-latam. Revisa que la clonación haya funcionado.

In [ ]:
print(f"DATA_DIR is: {DATA_DIR}") # Added print statement to check the path
csv_files = sorted([p for p in DATA_DIR.glob('**/*.csv')])
print('\nArchivos CSV detectados:')
for f in csv_files:
    print(' -', f.relative_to(Path.cwd()))

tables = {}
for f in csv_files:
    try:
        df = pd.read_csv(f, low_memory=False)
        tables[f.stem] = df
    except Exception as e:
        print(f'No se pudo leer {f.name}: {e}')

print('\nTablas cargadas:')
for name, df in tables.items():
    print(f' - {name}: {df.shape[0]} filas x {df.shape[1]} columnas')

print('\nPrimeras columnas de cada tabla:')
for name, df in tables.items():
    print('\n==', name, '==')
    display(df.head(2))

In [ ]:
candidates = {name: df for name, df in tables.items()}

def find_table_with_column(colname):
    for name, df in candidates.items():
        if colname in df.columns:
            return name, df
    return None, None

ord_items_name, ord_items = find_table_with_column('order_id')
products_name, products = find_table_with_column('product_id')
sellers_name, sellers = find_table_with_column('seller_id')
customers_name, customers = find_table_with_column('customer_id')
reviews_name, reviews = find_table_with_column('review_score')

print('\nMapeo probable de tablas:')
print(' order items ->', ord_items_name)
print(' products    ->', products_name)
print(' sellers     ->', sellers_name)
print(' customers   ->', customers_name)
print(' reviews     ->', reviews_name)



In [23]:
def choose_col(df, choices):
    """Devuelve el primer nombre de columna disponible en choices para el dataframe df."""
    for c in choices:
        if c in df.columns:
            return c
    return None

col_choices = {
    'order_id': ['order_id', 'order_id_pk', 'id_order'],
    'order_item_id': ['order_item_id', 'order_item_id_pk', 'item_id'],
    'product_id': ['product_id', 'id_product', 'product_id_pk'],
    'seller_id': ['seller_id', 'seller_id_pk', 'store_id', 'seller_id'],
    'price': ['price', 'price_value', 'payment_value', 'unit_price', 'order_item_price'],
    'freight_value': ['freight_value', 'shipping_cost', 'freight_value', 'shipping_fee'],
    'quantity': ['quantity', 'order_item_quantity', 'qty', 'items_quantity'],
    'product_category_name': ['product_category_name', 'category_name', 'category', 'product_category'],
    'review_score': ['review_score', 'score', 'review_rating'],
    'customer_id': ['customer_id', 'buyer_id', 'customer_id_pk']
}

colmap = {}

if ord_items is not None:
    df = ord_items
    for key, choices in col_choices.items():
        found = choose_col(df, choices)
        if found:
            colmap[key] = (ord_items_name, found)

for key, choices in col_choices.items():
    if key in colmap:
        continue
    for name, df in candidates.items():
        found = choose_col(df, choices)
        if found:
            colmap[key] = (name, found)
            break

print('\nColumn mapping detectado (tabla, columna):')
for k, v in colmap.items():
    print(f' - {k}: {v}')

if ord_items is None:
    raise RuntimeError('No fue posible encontrar una tabla con order_id. Revisa los CSV cargados.')

master = ord_items.copy()

if 'price' in colmap and colmap['price'][0] != ord_items_name:
    src_name, src_col = colmap['price']
    price_df = candidates[src_name][[colmap['product_id'][1] if 'product_id' in colmap else 'product_id', src_col]].drop_duplicates()
    master = master.merge(price_df, how='left', left_on=choose_col(master, [colmap['product_id'][1]]) , right_on=colmap['product_id'][1])
 rename_map = {}
for logical, (tbl, col) in colmap.items():

 if tbl == ord_items_name:
        rename_map[col] = logical

master = master.rename(columns=rename_map)

for logical, (tbl, col) in colmap.items():
    if logical in master.columns:
        continue

 if logical == 'product_category_name' and products is not None:
        key = colmap.get('product_id', (None, 'product_id'))[1]
        if key in master.columns and key in products.columns:
            master = master.merge(products[[key, col]], how='left', left_on=key, right_on=key)
            master = master.rename(columns={col: logical})
    elif logical == 'review_score' and reviews is not None:
 key = colmap.get('order_id', (None, 'order_id'))[1]
        if key in master.columns and key in reviews.columns:
            master = master.merge(reviews[[key, col]], how='left', left_on=key, right_on=key)
            master = master.rename(columns={col: logical})
    elif logical == 'seller_id' and sellers is not None:
  pass
    else:
  if col in candidates[tbl].columns:
 common = set(master.columns).intersection(candidates[tbl].columns)
            if common:
                common_key = list(common)[0]
                master = master.merge(candidates[tbl][[common_key, col]], how='left', on=common_key)
                master = master.rename(columns={col: logical})

print('\nMaster dataframe shape:', master.shape)
print('Columnas del master (muestra):')
print(master.columns.tolist())


for c in ['price', 'freight_value', 'quantity', 'review_score']:
    if c in master.columns:
        master[c] = pd.to_numeric(master[c], errors='coerce')

if 'quantity' not in master.columns:
    master['quantity'] = 1


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 52)

In [ ]:
def facturacion_por_tienda(master_df, seller_col_candidates=['seller_id','store_id','seller']):
    """Calcula facturación por tienda/vendedor. Retorna DataFrame con columnas ['store', 'revenue']"""
    seller_col = None
    for sc in seller_col_candidates:
        if sc in master_df.columns:
            seller_col = sc
            break
    if seller_col is None:
        raise RuntimeError('No se encontró una columna de tienda/vendedor en el dataframe maestro. Revisa nombres de columnas.')
    if 'price' not in master_df.columns:
        raise RuntimeError('No se encontró la columna price para calcular facturación.')

    master_df['revenue_line'] = master_df['price'] * master_df['quantity']
    revenue = master_df.groupby(seller_col)['revenue_line'].sum().reset_index().rename(columns={seller_col: 'store', 'revenue_line': 'revenue'})
    revenue = revenue.sort_values('revenue', ascending=False).reset_index(drop=True)
    return revenue


def categorias_populares_por_tienda(master_df, seller_col='seller_id', cat_col='product_category_name', topn=3):
    if seller_col not in master_df.columns or cat_col not in master_df.columns:
        raise RuntimeError('Columnas requeridas para categorías no presentes en master.')
    df = master_df.copy()
    df['sold_qty'] = df['quantity']
    grp = df.groupby([seller_col, cat_col])['sold_qty'].sum().reset_index()

tops = grp.sort_values(['seller_id', 'sold_qty'], ascending=[True, False]).groupby(seller_col).head(topn)
    return tops

def avg_review_por_tienda(master_df, seller_col='seller_id', review_col='review_score'):
    if seller_col not in master_df.columns or review_col not in master_df.columns:
        raise RuntimeError('Columnas requeridas para reviews no presentes en master.')
    return master_df.groupby(seller_col)[review_col].mean().reset_index().rename(columns={seller_col: 'store', review_col: 'avg_review'}).sort_values('avg_review', ascending=False)


def productos_mas_menos_vendidos(master_df, seller_col='seller_id', product_col='product_id', topn=1):
    if seller_col not in master_df.columns or product_col not in master_df.columns:
        raise RuntimeError('Columnas requeridas para productos no presentes en master.')
    df = master_df.copy()
    df['sold_qty'] = df['quantity']
    grp = df.groupby([seller_col, product_col])['sold_qty'].sum().reset_index()
    most = grp.sort_values(['seller_id','sold_qty'], ascending=[True, False]).groupby(seller_col).head(topn)
    least = grp.sort_values(['seller_id','sold_qty'], ascending=[True, True]).groupby(seller_col).head(topn)
    return most, least


def costo_promedio_envio(master_df, seller_col='seller_id', freight_col='freight_value'):
    if seller_col not in master_df.columns or freight_col not in master_df.columns:
        raise RuntimeError('Columnas requeridas para envío no presentes en master.')
    return master_df.groupby(seller_col)[freight_col].mean().reset_index().rename(columns={seller_col: 'store', freight_col: 'avg_freight'}).sort_values('avg_freight')

In [24]:
seller_candidates = ['seller_id','seller','store_id','store']
found_seller_col = None
for s in seller_candidates:
    if s in master.columns:
        found_seller_col = s
        break

if found_seller_col is None:
    raise RuntimeError('No se encontró ninguna columna de tienda/vendedor en el dataframe maestro. Revisa las tablas cargadas y sus nombres de columna.')

print(f'Usando columna de tienda: {found_seller_col}')


try:
    revenue = facturacion_por_tienda(master, seller_col_candidates=[found_seller_col])
    print('\nFacturación total por tienda:')
    display(revenue)
    revenue.plot(kind='bar', x='store', y='revenue', legend=False, title='Facturación total por tienda')
    plt.ylabel('Revenue')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('No se pudo calcular facturación:', e)

cat_col = None
for possible in ['product_category_name','category_name','category','product_category']:
    if possible in master.columns:
        cat_col = possible
        break
if cat_col is None:
    print('No se encontró columna de categoría de producto en el master. Saltando análisis de categorías.')
else:
    try:
        tops = categorias_populares_por_tienda(master, seller_col=found_seller_col, cat_col=cat_col, topn=5)
        print('\nCategorías más populares por tienda (top 5 por tienda):')
        display(tops)
    except Exception as e:
        print('Error al calcular categorías populares:', e)

if 'review_score' in master.columns:
    try:
        avg_rev = avg_review_por_tienda(master, seller_col=found_seller_col, review_col='review_score')
        print('\nPromedio de evaluación por tienda:')
        display(avg_rev)
    except Exception as e:
        print('No se pudo calcular promedio de reviews:', e)
else:
    print('\nNo se encontró columna review_score en master. Si existe en otra tabla, puedes unirla manualmente.')

prod_col = None
for possible in ['product_id','product_id_pk','product_code']:
    if possible in master.columns:
        prod_col = possible
        break
if prod_col is None:
    print('\nNo se encontró columna product_id en el master. Saltando productos más/menos vendidos.')
else:
    try:
        most, least = productos_mas_menos_vendidos(master, seller_col=found_seller_col, product_col=prod_col, topn=3)
        print('\nProductos más vendidos (top 3 por tienda):')
        display(most)
        print('\nProductos menos vendidos (top 3 por tienda):')
        display(least)
    except Exception as e:
        print('Error al calcular productos más/menos vendidos:', e)

if 'freight_value' in master.columns or 'shipping_cost' in master.columns:
    freight_col = 'freight_value' if 'freight_value' in master.columns else 'shipping_cost'
    try:
        avg_freight = costo_promedio_envio(master, seller_col=found_seller_col, freight_col=freight_col)
        print('\nCosto promedio de envío por tienda:')
        display(avg_freight)
    except Exception as e:
        print('Error al calcular costo promedio de envío:', e)
else:
    print('\nNo se encontró columna de costo de envío en el master. Si existe en otra tabla, puedes unirla manualmente.')


NameError: name 'master' is not defined

In [ ]:
print('\n--- Paso siguiente: Interpretación y recomendación ---')
print(textwrap.dedent('''
Con los resultados producidos arriba, identifique la tienda con menor desempeño considerando:
 - Facturación: la tienda con menor revenue total.
 - Evaluación: tiendas con review promedio bajo (posible riesgo reputacional).
 - Popularidad: tiendas con pocas categorías o ventas concentradas en pocos productos.
 - Costo de envío: tiendas con costo de envío promedio alto que erosionen margen.

Recomendación sugerida: vender la tienda que esté consistentemente en los peores puestos en varias de estas métricas.

Si quieres, puedo generar un informe automatizado en formato PDF o PowerPoint con tablas, gráficos y recomendaciones ejecutivas.
'''))